In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# in-cell imports:
# import warnings
# from scipy import stats
# from scipy import spatial
# import pathlib
# from pathlib import Path

%matplotlib inline

# Load & subset data

In [2]:
adata = ad.read_h5ad("../data/merfish_609882and638850_AIT17_annotated_TH_ZI_only_shared_genes_only_2023-04-07_00-00-00/Brain_1_3_TH_ZI.h5ad")

In [3]:
# get a df of the cell-by-gene table
adata_df = adata.to_df(layer='raw')

# sort the columns in alphabetical order so that it's easier to manually check flagged genes
adata_df = adata_df.reindex(sorted(adata_df.columns), axis=1)

## gene lists

In [4]:
# lists of (1) all genes, excluding blanks ...
gene_names = adata_df.filter(regex='^(?!(Blank))').columns.to_list() # blanks have the naming convention "Blank-###"
print(len(gene_names), 'genes shared between brain1 & brain3\'s gene panels', )
print(gene_names)

# ... and (2) just blanks
#   Note that because brain1 & brain3 have different codebooks AND different 
#   number of blanks (brain1=200, brain3=50), blanks with the same "Blank-###" 
#   label should be considered the same *in name only* and not the same in terms
#   of counts, etc.
blank_names = adata_df.filter(regex='Blank').columns.to_list() # blanks have the naming convention "Blank-###"

## subset by brain

In [5]:
# view obs columns that might allow for easy filtering of brain1 vs brain3
#  - animal *should* be different, but isn't, likely due to weird anndata 
#      behavior when concatenating brain1 & brain3 objects
#  - using codebook going forward since it's the one I'm next-most-confident 
#      should be consistent within each brain
adata.obs[['section', 'animal', 'merscope', 'target_atlas_plate', 'codebook']]

In [6]:
# Distinguish between the two brains by using the codebook names
brain1_codebook = 'VZG147'
brain3_codebook = 'wholebrain031822a'

# Generate separate df for each brain, and for actual genes vs blanks
brain1_genes_df = adata_df[gene_names][adata.obs['codebook']==brain1_codebook]
brain1_blanks_df = adata_df[blank_names][adata.obs['codebook']==brain1_codebook]
brain1_obs = adata.obs[adata.obs['codebook']==brain1_codebook]

brain3_genes_df = adata_df[gene_names][adata.obs['codebook']==brain3_codebook]
brain3_blanks_df = adata_df[blank_names][adata.obs['codebook']==brain3_codebook]
brain3_obs = adata.obs[adata.obs['codebook']==brain3_codebook]

### define division & subclass filters

In [7]:
# Division labels for just the neurons in the thalamus
#   NB: division_id_label is consistent between the two taxonomy mappings,
#       AIT17.0 and AIT17.custom, used for brain1 and brain3, whereas 
#       class_id_label may *look* similar to division labels but are NOT 
#       necessarily consistent across brains
divisions_neuron = [
                    '1 Pallium glutamatergic',
                    '2 Subpallium GABAergic',
                    '3 PAL-sAMY-TH-HY-MB-HB neuronal',
                    '4 CBX-MOB-other neuronal'
                   ]

# Use subclass_id_label to distinguish between excitatory (exc) and 
# inhibitory (inh) cells
#   NB: subclass_id_label is NOT fully consistent between the two taxonomy
#       mappings, AIT17.0 and AIT17.custom, used for brain1 and brain3, 
#       respectively, so be cautious using anything more specific than 'glut'
#       vs 'gaba' filtering
exc_cell_str = 'glut'
inh_cell_str = 'gaba'

In [8]:
# Create brain-specific filters for just the neuronal cells, using the division labels
neuron_filter_brain1 = brain1_obs['division_id_label'].isin(divisions_neuron)
neuron_filter_brain3 = brain3_obs['division_id_label'].isin(divisions_neuron)

# Filter for excitatory (exc) neurons vs inhibitory (inh) neurons, using the 
# subclass strings + neuron_filter defined above
exc_neuron_filter_brain1 = brain1_obs['subclass_id_label'].str.contains(exc_cell_str,case=False) & neuron_filter_brain1
inh_neuron_filter_brain1 = brain1_obs['subclass_id_label'].str.contains(inh_cell_str,case=False) & neuron_filter_brain1

exc_neuron_filter_brain3 = brain3_obs['subclass_id_label'].str.contains(exc_cell_str,case=False) & neuron_filter_brain3
inh_neuron_filter_brain3 = brain3_obs['subclass_id_label'].str.contains(inh_cell_str,case=False) & neuron_filter_brain3

# Compare mean gene reads in brain1 vs brain3

In [9]:
# Create a df of all the genes so that we can easily compare all the flags 
# we're about to generate
gene_flags_df = pd.DataFrame(index=gene_names)

In [10]:
# Function to generate a scatter plot of gene-by-gene comparisons between
# brain1 & brain3
def plot_brain1_vs_brain3_reads(gene_reads_brain1, gene_reads_brain3,
                                blank_reads_brain1, blank_reads_brain3,
                                gene_names, subset, **kwargs):
    
    # TODO: add type-enforcement?
    genes_to_highlight = kwargs.get('genes_to_highlight', None) # string or list of strings
    
    fig, ax = plt.subplots(figsize=(11,9))

    # Scatter plot of brain1 vs brain3 reads
    ax.scatter(gene_reads_brain1, gene_reads_brain3, 
               s=18, color='cornflowerblue')
    # label each data point with the gene name, using a non-black color to 
    # highlight user-specified genes, if they exist
    if genes_to_highlight is None:
        for i, gene in enumerate(gene_names):
            ax.annotate(gene_names[i], 
                        (gene_reads_brain1[i], gene_reads_brain3[i]), 
                        fontsize=10, color='k')
    else:
        for i, gene in enumerate(gene_names):
            if gene in genes_to_highlight:
                ax.annotate(gene_names[i], 
                            (gene_reads_brain1[i], gene_reads_brain3[i]), 
                            fontsize=16, color='magenta') 
            else:
                ax.annotate(gene_names[i], 
                            (gene_reads_brain1[i], gene_reads_brain3[i]), 
                            fontsize=10)

    # Plot the mean blank reads thresholds as a vertical line (brain1) and a
    # horizontal line (brain3)
    vline_ymin = min(gene_reads_brain3) # vert line limits
    vline_ymax = max(gene_reads_brain3) # vert line limits
    hline_xmin = min(gene_reads_brain1) # hor line limits
    hline_xmax = max(gene_reads_brain1) # hor line limits
    ax.vlines(blank_reads_brain1, ymin=vline_ymin, ymax=vline_ymax, 
              color='darkorange', ls='--')
    ax.hlines(blank_reads_brain3, xmin=hline_xmin,xmax=hline_xmax, 
              color='darkorange', ls='--')
    ax.annotate('brain1\nmean blank reads\nper '+subset, 
                (blank_reads_brain1+0.005,10), color='darkorange', fontsize=12)
    ax.annotate('brain3\nmean blank reads\nper '+subset, 
                (3, blank_reads_brain3+0.01), color='darkorange', fontsize=12)

    # Plot a slope=1 line for visual reference of perfect brain1 vs brain3 match
    ax.axline((1,1),slope=1, ls='-',c='darkgrey')

    # Axis formatting
    # ax.set_aspect('equal')
    ax.set_xlabel('brain1\n'+'mean gene reads per '+subset)
    ax.set_ylabel('brain3\n'+'mean gene reads per '+subset)
    ax.tick_params('both', which='both', direction='in', top=True, right=True,
                   color='grey')
    ax.spines['bottom'].set_color('grey')
    ax.spines['top'].set_color('grey') 
    ax.spines['right'].set_color('grey')
    ax.spines['left'].set_color('grey')
    ax.set_xscale('log')
    ax.set_yscale('log')
    # special function needed to get the major tick labels to display correctly
    # on log axes in non-scientific number format
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))

    plt.show()

## In all thalamic cells

In [11]:
# For all cells
subset = 'cell'

# Calculate mean reads per cell, for each gene
mean_gene_reads_per_cell_brain1 = brain1_genes_df.mean(axis=0)
mean_gene_reads_per_cell_brain3 = brain3_genes_df.mean(axis=0)

# Calculate overall mean of the mean blank reads per cell
mean_blank_reads_per_cell_brain1 = np.mean(brain1_blanks_df.mean(axis=1))
mean_blank_reads_per_cell_brain3 = np.mean(brain3_blanks_df.mean(axis=1))

# Plot
plot_brain1_vs_brain3_reads(mean_gene_reads_per_cell_brain1, 
                            mean_gene_reads_per_cell_brain3,
                            mean_blank_reads_per_cell_brain1,
                            mean_blank_reads_per_cell_brain3,
                            gene_names, subset, genes_to_highlight=None) #genes_to_highlight=['Npsr1', 'Gad2'])

# Flag & store genes that have mean reads in neurons below the overall mean blank read
gene_flags_df['blanks_all_cells_brain1'] = mean_gene_reads_per_cell_brain1 <= mean_blank_reads_per_cell_brain1
gene_flags_df['blanks_all_cells_brain3'] = mean_gene_reads_per_cell_brain3 <= mean_blank_reads_per_cell_brain3

## In neurons

In [12]:
# For only neurons
subset = 'neuron'

# Calculate mean reads per cell, for each gene
mean_gene_reads_per_neuron_brain1 = brain1_genes_df[neuron_filter_brain1].mean(axis=0)
mean_gene_reads_per_neuron_brain3 = brain3_genes_df[neuron_filter_brain3].mean(axis=0)

# Calculate overall mean of the mean blank reads per cell
mean_blank_reads_per_neuron_brain1 = np.mean(brain1_blanks_df[neuron_filter_brain1].mean(axis=1))
mean_blank_reads_per_neuron_brain3 = np.mean(brain3_blanks_df[neuron_filter_brain3].mean(axis=1))

# Plot - uncomment below to view
# plot_brain1_vs_brain3_reads(mean_gene_reads_per_neuron_brain1, 
#                             mean_gene_reads_per_neuron_brain3,
#                             mean_blank_reads_per_neuron_brain1,
#                             mean_blank_reads_per_neuron_brain3,
#                             gene_names, subset)

# Flag & store genes that have mean reads in neurons below the overall mean blank read
gene_flags_df['blanks_neuron_brain1'] = mean_gene_reads_per_neuron_brain1 <= mean_blank_reads_per_neuron_brain1
gene_flags_df['blanks_neuron_brain3'] = mean_gene_reads_per_neuron_brain3 <= mean_blank_reads_per_neuron_brain3

## In non-neurons

In [13]:
# For only non-neuronal cells
subset = 'non-neuronal cell'

# Calculate mean reads per cell, for each gene
mean_gene_reads_per_nn_brain1 = brain1_genes_df[~neuron_filter_brain1].mean(axis=0)
mean_gene_reads_per_nn_brain3 = brain3_genes_df[~neuron_filter_brain3].mean(axis=0)

# Calculate overall mean of the mean blank reads per cell
mean_blank_reads_per_nn_brain1 = np.mean(brain1_blanks_df[~neuron_filter_brain1].mean(axis=1))
mean_blank_reads_per_nn_brain3 = np.mean(brain3_blanks_df[~neuron_filter_brain3].mean(axis=1))

# Plot - uncomment below to view
# plot_brain1_vs_brain3_reads(mean_gene_reads_per_nn_brain1, 
#                             mean_gene_reads_per_nn_brain3,
#                             mean_blank_reads_per_nn_brain1,
#                             mean_blank_reads_per_nn_brain3,
#                             gene_names, subset)

# Flag & store genes that have mean reads in non-neuronal cells below the overall mean blank read
gene_flags_df['blanks_nn_brain1'] = mean_gene_reads_per_nn_brain1 <= mean_blank_reads_per_nn_brain1
gene_flags_df['blanks_nn_brain3'] = mean_gene_reads_per_nn_brain3 <= mean_blank_reads_per_nn_brain3

## In excitatory neurons

In [14]:
# For only excitatory neurons
subset = 'excitatory neuron'

# Calculate mean reads per cell, for each gene
mean_gene_reads_per_exc_brain1 = brain1_genes_df[exc_neuron_filter_brain1].mean(axis=0)
mean_gene_reads_per_exc_brain3 = brain3_genes_df[exc_neuron_filter_brain3].mean(axis=0)

# Calculate overall mean of the mean blank reads per cell
mean_blank_reads_per_exc_brain1 = np.mean(brain1_blanks_df[exc_neuron_filter_brain1].mean(axis=1))
mean_blank_reads_per_exc_brain3 = np.mean(brain3_blanks_df[exc_neuron_filter_brain3].mean(axis=1))

# Plot - uncomment below to view
# plot_brain1_vs_brain3_reads(mean_gene_reads_per_exc_brain1, 
#                             mean_gene_reads_per_exc_brain3,
#                             mean_blank_reads_per_exc_brain1,
#                             mean_blank_reads_per_exc_brain3,
#                             gene_names, subset)

# Flag & store genes that have mean reads in exc cells below the overall mean blank read
gene_flags_df['blanks_exc_brain1'] = mean_gene_reads_per_exc_brain1 <= mean_blank_reads_per_exc_brain1
gene_flags_df['blanks_exc_brain3'] = mean_gene_reads_per_exc_brain3 <= mean_blank_reads_per_exc_brain3

## In inhibitory neurons

In [15]:
# For only inhibitory neurons
subset = 'inhibitory neuron'

# Calculate mean reads per cell, for each gene
mean_gene_reads_per_inh_brain1 = brain1_genes_df[inh_neuron_filter_brain1].mean(axis=0)
mean_gene_reads_per_inh_brain3 = brain3_genes_df[inh_neuron_filter_brain3].mean(axis=0)

# Calculate overall mean of the mean blank reads per cell
mean_blank_reads_per_inh_brain1 = np.mean(brain1_blanks_df[inh_neuron_filter_brain1].mean(axis=1))
mean_blank_reads_per_inh_brain3 = np.mean(brain3_blanks_df[inh_neuron_filter_brain3].mean(axis=1))

# Plot - uncomment below to view
# plot_brain1_vs_brain3_reads(mean_gene_reads_per_inh_brain1, 
#                             mean_gene_reads_per_inh_brain3,
#                             mean_blank_reads_per_inh_brain1,
#                             mean_blank_reads_per_inh_brain3,
#                             gene_names, subset)

# Flag & store genes that have mean reads in inh cells below the overall mean blank read
gene_flags_df['blanks_inh_brain1'] = mean_gene_reads_per_inh_brain1 <= mean_blank_reads_per_inh_brain1
gene_flags_df['blanks_inh_brain3'] = mean_gene_reads_per_inh_brain3 <= mean_blank_reads_per_inh_brain3

## Which genes were flagged for below-blank reads?

In [16]:
blank_flagged_all_genes = gene_flags_df.index[gene_flags_df.sum(axis=1)>0]

# Note: all blank-flagged genes were flagged in both brains by one of the thresholds, but I'm keeping these just in case I want to confirm/show that
blank_flagged_brain1_genes = gene_flags_df[gene_flags_df.filter(regex='brain1').columns].index[gene_flags_df.sum(axis=1)>0]
blank_flagged_brain3_genes = gene_flags_df[gene_flags_df.filter(regex='brain3').columns].index[gene_flags_df.sum(axis=1)>0]

blank_flagged_brain1 = sorted(gene_flags_df[gene_flags_df['blanks_all_cells_brain1']==1].index)
blank_flagged_brain3 = sorted(gene_flags_df[gene_flags_df['blanks_all_cells_brain3']==1].index)
blank_flagged_both = sorted(list(set(blank_flagged_brain1).intersection(set(blank_flagged_brain3))))
blank_flagged_brain1only = sorted(list(set(blank_flagged_brain1) - (set(blank_flagged_brain3))))
blank_flagged_brain3only = sorted(list(set(blank_flagged_brain3) - (set(blank_flagged_brain1))))

print('Genes with low avg expression in either brain, any subset of cells:', blank_flagged_all_genes.to_list())
print('\nGenes with low avg expression in Brain 1 only: ', blank_flagged_brain1only)
print('Genes with low avg expression in Brain 3 only: ', blank_flagged_brain3only)
print('Genes with low avg expression in both brains: ', blank_flagged_both)

# Variance, CV vs mean for brain1 & brain3

In [17]:
fig, axes = plt.subplots(1,2, figsize=(20,6))
b1_mean = brain1_genes_df.mean()
b1_var = brain1_genes_df.var()
b1_std = brain1_genes_df.std()

ax = axes[0]
ax.scatter(brain1_genes_df.mean(), brain1_genes_df.var(), s=10, color='b', label='brain 1') #c=np.arange(len(brain_1_genes_df.mean())), cmap='Spectral', marker='o')
ax.scatter(brain3_genes_df.mean(), brain3_genes_df.var(), s=10, color='r', label='brain 3') # c=np.arange(len(brain_3_genes_df.mean())), cmap='Spectral', marker='^')
for i, gene in enumerate(gene_names):
    ax.annotate(gene_names[i], (b1_mean[i], b1_var[i]))
ax.set_xlabel('gene reads mean')
ax.set_ylabel('gene reads variance')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()

ax = axes[1]
ax.scatter(brain1_genes_df.mean(), brain1_genes_df.std()/brain1_genes_df.mean(), s=10, color='b', label='brain 1')
ax.scatter(brain3_genes_df.mean(), brain3_genes_df.std()/brain3_genes_df.mean(), s=10, color='r', label='brain 3')
for i, gene in enumerate(gene_names):
    ax.annotate(gene_names[i], (b1_mean[i], b1_std[i]/b1_mean[i]))
ax.set_xlabel('gene reads mean')
ax.set_ylabel('gene reads CV (std/mean)')
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.legend();

# Compare 95th, 99th, 99.9th percentile of gene reads in brain1 vs brain3

In [18]:
# Function to generate a scatter plot of gene-by-gene comparisons between
# brain1 & brain3
def plot_brain1_vs_brain3_percentile(gene_reads_pct_brain1, 
                                     gene_reads_pct_brain3,
                                     gene_names, percentile, **kwargs):
    
    # TODO: add type-enforcement?
    genes_to_highlight = kwargs.get('genes_to_highlight', None) # string or list of strings
    
    fig, ax = plt.subplots(figsize=(11,9))

    # Scatter plot of brain1 vs brain3 reads
    ax.scatter(gene_reads_pct_brain1, gene_reads_pct_brain3, 
               s=18, color='cornflowerblue')
    
    # label each data point with the gene name, using a non-black color to 
    # highlight user-specified genes, if they exist
    if genes_to_highlight is None:
        for i, gene in enumerate(gene_names):
            ax.annotate(gene_names[i], 
                        (gene_reads_pct_brain1[i], gene_reads_pct_brain3[i]), 
                        fontsize=10, color='k')
    else:
        for i, gene in enumerate(gene_names):
            if gene in genes_to_highlight:
                ax.annotate(gene_names[i], 
                            (gene_reads_pct_brain1[i], gene_reads_pct_brain3[i]), 
                            fontsize=16, color='magenta') 
            else:
                ax.annotate(gene_names[i], 
                            (gene_reads_pct_brain1[i], gene_reads_pct_brain3[i]), 
                            fontsize=10)

    # Plot a slope=1 line for visual reference of perfect brain1 vs brain3 match
    ax.axline((1,1),slope=1, ls='-',c='darkgrey')

    # Axis formatting
    # ax.set_aspect('equal')
    ax.set_xlabel('brain1\n'+str(percentile)+'th percentile of gene reads')
    ax.set_ylabel('brain3\n'+str(percentile)+'th percentile of gene reads')
    ax.tick_params('both', which='both', direction='in', top=True, right=True,
                   color='grey')
    ax.spines['bottom'].set_color('grey')
    ax.spines['top'].set_color('grey') 
    ax.spines['right'].set_color('grey')
    ax.spines['left'].set_color('grey')
    # ax.set_xscale('log')
    # ax.set_yscale('log')
    # # special function needed to get the major tick labels to display correctly
    # # on log axes in non-scientific number format
    # ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    # ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: '{:g}'.format(x)))

    plt.show()

In [85]:
# Plot the 95th percentile of reads for brain1 vs brain3
# Are any gene egregiously off the diagonal?
pct = 0.95
gene_95pct_brain1 = brain1_genes_df.quantile(pct)
gene_95pct_brain3 = brain3_genes_df.quantile(pct)

plot_brain1_vs_brain3_percentile(gene_95pct_brain1, gene_95pct_brain3, 
                                 gene_names, int(pct*100), 
                                 genes_to_highlight=None)

In [20]:
# Plot the 99th percentile of reads for brain1 vs brain3
# Are any gene egregiously off the diagonal?
pct = 0.99
gene_99pct_brain1 = brain1_genes_df.quantile(pct)
gene_99pct_brain3 = brain3_genes_df.quantile(pct)

plot_brain1_vs_brain3_percentile(gene_99pct_brain1, gene_99pct_brain3, 
                                 gene_names, int(pct*100), 
                                 genes_to_highlight=None)

In [21]:
# Plot the 99.9th percentile of reads for brain1 vs brain3
# Are any gene egregiously off the diagonal?
pct = 0.999
gene_99pt9pct_brain1 = brain1_genes_df.quantile(pct)
gene_99pt9pct_brain3 = brain3_genes_df.quantile(pct)

plot_brain1_vs_brain3_percentile(gene_99pt9pct_brain1, gene_99pt9pct_brain3, 
                                 gene_names, pct*100, 
                                 genes_to_highlight=None)

# Mean-difference (MD) plots

aka Bland-Altman plot, aka Tukey mean-difference plot, typically used for determining variance or inconsistencies between two measurement methods in fields like analytical chemistry & clinical laboratories

x-axis: average of two methods of measuring the same underlying values

y-axis: difference between the two methods

In [141]:
import warnings
from scipy import stats

# Function to check if the difference between brain1 & brain3 values is normally
# distributed
def is_difference_distribution_normal(measurement1, measurement2,
                                      zscore_thresh=1.96, data_transform=None):
    """
    Checks if the difference between two measurement sets is normally 
    distributed.

    The applicability of the Limits of Agreement (LoA) for a Bland-Altman 
    mean-difference (MD) plot are based on the assumption that the differences
    between the two measurements are normally distribution, so it's imperative
    to check for normality before interpreting Bland-Altman plots

    Parameters
    ----------
    measurement1 : array-like
        Array of numbers from one measurement of the underlying values
    measurement2 : array-like
        Array of numbers from another, different, measurement of the underlying 
        values
    zscore_thresh : float (default:1.96)
    data_transform : {'log', None, 'none'} (default: None)
        Transformation permformed on measurement1 and measurement2 prior to
        calculations for and generation of the MD plot. Possible values:
        - 'log': natural log transform of the measurements
        - 'none', None: no transform

    Returns
    -------
    figure
        Histogram and best fit normal curve
    skew_zscore
    kurtosis_zscore
    normal_stat
    """
    # Calculate difference between measurements
    if data_transform=='log':
        measurement_differences = np.log(measurement1) - np.log(measurement2)
        # clean up to remove -inf from log(0) genes and replace with nan so no error is thrown
        measurement_differences[(measurement_differences==np.inf) | 
                                (measurement_differences==-np.inf)] = np.nan
        diff_for_norm_fit = measurement_differences[~np.isnan(measurement_differences)]
    else:
        measurement_differences = measurement1 - measurement2
        diff_for_norm_fit = measurement_differences

    # Plot difference distribution & normal fit
    fig, ax = plt.subplots(figsize=(6,4))
    
    _, bins, _ = ax.hist(measurement_differences, bins=20, density=1,
                         color='cornflowerblue')
    
    mu, sigma = stats.norm.fit(diff_for_norm_fit)
    best_fit_line = stats.norm.pdf(bins, mu, sigma)
    ax.plot(bins, best_fit_line, color='darkorange')
    
    ax.set_ylabel('counts')
    if data_transform!=None:
        ax.set_xlabel(data_transform+'(difference)')
    else:
        ax.set_xlabel('difference')

    # Explicitly test & display whether or not the difference distribution 
    # matches a normal distribution
    skew_zscore, skew_pval = stats.skewtest(measurement_differences, nan_policy='omit')
    if skew_zscore > zscore_thresh:
        print('skew test zscore =', '%0.2f'%skew_zscore, 
              'indicates difference distribution has a positive skew and is not normally distributed')
    elif skew_zscore < -zscore_thresh:
        print('skew test zscore =', '%0.2f'%skew_zscore, 
              'indicates difference distribution has a negative skew and is not normally distributed')
    else:
        print('skew test zscore =', '%0.2f'%skew_zscore, 
              'indicates difference distribution has no skew and not significantly different from a normal distribution')

    kurtosis_zscore, kurtosis_pval = stats.kurtosistest(measurement_differences, nan_policy='omit')
    if kurtosis_zscore > zscore_thresh:
        print('kurtosis test zscore =', '%0.2f'%kurtosis_zscore, 
              'indicates difference distribution is more peaked than a normal distribution')
    elif kurtosis_zscore < -zscore_thresh:
        print('kurtosis test zscore =', '%0.2f'%kurtosis_zscore, 
              'indicates difference distribution is less peaked than a normal distribution')
    else:
        print('kurtosis test zscore =', '%0.2f'%kurtosis_zscore,
              'indicates difference distribution is not significantly different from a normal distribution')

    normal_stat, normal_pval = stats.normaltest(measurement_differences, nan_policy='omit')
    if normal_stat > zscore_thresh:
        print('normal test score =', '%0.2f'%normal_stat,
              'indicates difference distribution significantly deviates from a normal distribution')
    else:
        print('normal test score =','%0.2f'%normal_stat,
              'indicates difference distribution does not significantly deviate from a normal distribution')
        
    return skew_zscore, kurtosis_zscore, normal_stat
    

# Function to generate a MD plot
def generate_mean_difference_plot(measurement1, measurement2,
                                  measurement1_label='measurement1', 
                                  measurement2_label='measurement2',
                                  loa_std=2.0, data_transform=None,
                                  title=None):
    """
    Generates a mean-difference plot from two measurements of the (assumed) same
    underlying values.

    Also sometimes called a Bland-Altman plot or a Tukey mean-difference plot.

    Parameters
    ----------
    measurement1 : array-like
        Array of numbers from one measurement of the underlying values
    measurement2 : array-like
        Array of numbers from another, different, measurement of the underlying 
        values
    measurement1_label : str (default: 'measurement1')
        The label for measurement1 data, used to annotate the plot axes
    measurement1_label : str (default: 'measurement2')
        The label for measurement2 data, used to annotate the plot axes
    loa_std : float or int, positive (default: 2.0)
        Limits of agreement (LoA), specified in standard deviations from the
        mean, and used as a threshold to determine outlying data points.
        Default is 2.0 standard deviations from the mean, which in a *normal
        distribution* encompasses 95.5% of the data (1.0std=68.3%; 3.0std=99.7%)
    data_transform : {'log', None, 'none'} (default: None)
        Transformation permformed on measurement1 and measurement2 prior to
        calculations for and generation of the MD plot. Possible values:
        - 'log': natural log transform of the measurements
        - 'diff_as_pct_of_mean': 
        - 'none', None: no transform

    Returns
    -------
    outlier_genes
        List of genes flagged as outliers
    figure
        Annotated mean-difference (MD) plot
    """
    loa_std 
    
    fig, ax = plt.subplots(figsize=(11,6))
    
    # Calculate the mean and difference of the two measurements
    # transform the measurements first, if specified
    if data_transform=='log':
        # clearn up zero entries & warn user
        if np.any(measurement1==0) | np.any(measurement2==0):
            warnings.warn('\nOne or more zero values detected in measurement1 '+
                          'or measurement2. \nThese data points will be '+
                          'excluded from the final MD plot because they '+
                          'result in a divide-by-zero error in the log '+
                          'transform. \nSet data_transform=None if you wish '+
                          'to view all data points.')
            measurement1[(measurement1==0) | (measurement1==0)] = np.nan
            measurement2[(measurement2==0) | (measurement2==0)] = np.nan
        
        measurement_diff = np.log(measurement1) - np.log(measurement2)
        measurement_mean = (np.log(measurement1) + np.log(measurement2)) / 2
        
    elif data_transform=='diff_as_pct_of_mean':
        measurement_mean = (measurement1 + measurement2) / 2
        measurement_diff = ((measurement1 - measurement2)/measurement_mean)*100
        
    elif (data_transform==None) | (data_transform=='none'):
        measurement_diff = measurement1 - measurement2
        measurement_mean = (measurement1 + measurement2) / 2
        
    # Calculate the mean & std of the difference to use for the LoA
    diff_mean = np.nanmean(measurement_diff)
    diff_std = np.nanstd(measurement_diff)

    # Generate the MD plot
    ax.scatter(measurement_mean, measurement_diff, s=18, color='cornflowerblue')
    for i, gene in enumerate(gene_names):
        ax.annotate(gene_names[i], (measurement_mean[i], measurement_diff[i]))

    # Show and label the mean & loa_std values
    annotate_ypos_offset = 0.015*np.nanmax(np.absolute(measurement_diff))
    annotate_xpos = 0.90*np.nanmax(measurement_mean)
    ax.axhline(diff_mean, color='grey', linestyle='--')
    ax.axhline(diff_mean + 1.96*diff_std, color='darkorange', linestyle='--')
    ax.axhline(diff_mean - 1.96*diff_std, color='darkorange', linestyle='--')
    ax.annotate('$\mu$ = %0.4f'%diff_mean, 
                (annotate_xpos,diff_mean+annotate_ypos_offset), 
                color='grey')
    ax.annotate('$\mu+$'+str(loa_std)+'$\sigma =$ %0.2f'%(diff_mean + loa_std*diff_std), 
                (annotate_xpos,(diff_mean + loa_std*diff_std+annotate_ypos_offset)), 
                color='darkorange')
    ax.annotate('$\mu+$'+str(loa_std)+'$\sigma =$ %0.2f'%(diff_mean - loa_std*diff_std), 
                (annotate_xpos,((diff_mean - loa_std*diff_std)+annotate_ypos_offset)), 
                color='darkorange')
    
    if (data_transform=='none') | (data_transform==None):
        ax.set_xlabel('mean( ('+measurement1_label+'), ('+measurement2_label+') )')
        ax.set_ylabel('('+measurement1_label+') - ('+measurement2_label+')')
    elif data_transform=='log':
        ax.set_xlabel('mean( '+data_transform+'('+measurement1_label+'), '+data_transform+'('+measurement2_label+') )')
        ax.set_ylabel(data_transform+'('+measurement1_label+') - '+data_transform+'('+measurement2_label+')')
    elif data_transform=='diff_as_pct_of_mean':
        ax.set_xlabel('mean( ('+measurement1_label+'), ('+measurement2_label+') )')
        ax.set_ylabel('(('+measurement1_label+')-('+measurement2_label+')) / mean \n(%)')
    
    ax.set_title(title)
    plt.show()
    
    # Generate a list of flagged genes (those outside the LoA)
    outlier_genes = measurement_diff[(measurement_diff > (diff_mean + loa_std*diff_std)) | 
                                     (measurement_diff < (diff_mean - loa_std*diff_std))].index
    return outlier_genes

## Standard MD plot

The standard MD plots may not actually be helpful because we are trying to compare data points spanning 3 orders of magnitude using the same scale.

So genes like Gad2, which we have visually confirmed to be fine in both datasets, end up falling almost outside of our confidence interval simply because they have high counts and so a small difference between the two datasets looks large in absolute terms, but is not that large compared to the mean.

On the other hand, it's hard to actually get a sense of how much variation there is for the genes with both small means and small differences because they're all smashed together.

One idea on how to handle this large span is to use log(95th pct gene reads, brain X) in the mean and difference calculations, which I do below. However, I haven't found any examples where this approach was used, nor do I claim that this has any sort of statistical rigor.

Another idea, taken from "Giavarina D. (2015). Biochemia Medica. 25(2): 141-51. doi:10.11613/BM.2015.015.", is to divide the difference by the mean to plot the difference as a percentage of the mean, which is "useful when there is an increase in variability of the difference as teh magnitude of the measurement increases"


In [108]:
pct = 0.95
gene_95pct_brain1 = brain1_genes_df.quantile(pct)
gene_95pct_brain3 = brain3_genes_df.quantile(pct)

skew_zscore, kurtosis_zscore, normal_stat = is_difference_distribution_normal(
                                                gene_95pct_brain1, 
                                                gene_95pct_brain3)

outlier_genes_95pct = generate_mean_difference_plot(gene_95pct_brain1,
                                                    gene_95pct_brain3, 
                                                    '95th pct gene reads, brain1', 
                                                    '95th pct gene reads, brain3',
                                                    loa_std=2.0, 
                                                    data_transform=None)

## Modified MD w/ log-transformed data

The read counts span 3 orders of magnitude, so calculating the mean and difference of the log-transformed 95th percentile of reads should make things more quantitatively comparable

HOWEVER, because the 95th percentile read for many genes is still 0, and log(0)=-inf, a number of genes are simply not represented in that plot at all.

The minimum read for 99th percentile is 1.0, so the log transform of that metric does not have the same problem

Due to the gene drop-out in the 95th percentile, it's best to only use this plot with the 99th percentile (or whatever percentile's minimus is >0)

In [104]:
# Don't use 95th pct in log-transformed plots due to gene drop-out when reads=0

# pct = 0.95
# gene_95pct_brain1 = brain1_genes_df.quantile(pct)
# gene_95pct_brain3 = brain3_genes_df.quantile(pct)

# skew_zscore, kurtosis_zscore, normal_stat = is_difference_distribution_normal(
#                                                 gene_95pct_brain1, 
#                                                 gene_95pct_brain3,
#                                                 data_transform='log')

# # MD plot of the log-transformed 95th percentile of gene reads
# outlier_genes_95pct =  generate_mean_difference_plot(gene_95pct_brain1, gene_95pct_brain3, 
#                                                       '95th pct gene reads, brain1', 
#                                                       '95th pct gene reads, brain3',
#                                                       loa_std=2, data_transform='log')

### MD of 99th pct in all cells

In [105]:
# MD plot of the log-transformed 99th percentile of gene reads
# for brain1 & brain3 all genes' 99th pct > 0
pct = 0.99
gene_99pct_brain1 = brain1_genes_df.quantile(pct)
gene_99pct_brain3 = brain3_genes_df.quantile(pct)

skew_zscore, kurtosis_zscore, normal_stat = is_difference_distribution_normal(
                                                gene_99pct_brain1, 
                                                gene_99pct_brain3,
                                                data_transform='log')

outlier_genes_99pct_log = generate_mean_difference_plot(gene_99pct_brain1,
                                                        gene_99pct_brain3, 
                                                        '99th pct gene reads, brain1', 
                                                        '99th pct gene reads, brain3',
                                                        loa_std=2, data_transform='log')

### MD of 99th pct in subsets (neurons, NN, exc, inh)

In [111]:
pct = 0.99

# neurons
gene_99pct_brain1_neurons = brain1_genes_df[neuron_filter_brain1].quantile(pct)
gene_99pct_brain3_neurons = brain3_genes_df[neuron_filter_brain3].quantile(pct)

outlier_genes_99pct_neurons = generate_mean_difference_plot(
                                    gene_99pct_brain1_neurons,
                                    gene_99pct_brain3_neurons, 
                                    '99th pct gene reads, brain1', 
                                    '99th pct gene reads, brain3',
                                    loa_std=2, data_transform='log', 
                                    title='neurons')

# non-neuronal cells
gene_99pct_brain1_nn = brain1_genes_df[~neuron_filter_brain1].quantile(pct)
gene_99pct_brain3_nn = brain3_genes_df[~neuron_filter_brain3].quantile(pct)

outlier_genes_99pct_nn = generate_mean_difference_plot(
                                    gene_99pct_brain1_nn,
                                    gene_99pct_brain3_nn, 
                                    '99th pct gene reads, brain1', 
                                    '99th pct gene reads, brain3',
                                    loa_std=2, data_transform='log', 
                                    title='non-neuronal cells')

# excitatory neurons
gene_99pct_brain1_exc = brain1_genes_df[exc_neuron_filter_brain1].quantile(pct)
gene_99pct_brain3_exc = brain3_genes_df[exc_neuron_filter_brain3].quantile(pct)

outlier_genes_99pct_exc = generate_mean_difference_plot(
                                    gene_99pct_brain1_exc,
                                    gene_99pct_brain3_exc, 
                                    '99th pct gene reads, brain1', 
                                    '99th pct gene reads, brain3',
                                    loa_std=2, data_transform='log', 
                                    title='excitatory neurons')

# inhibitory neurons
gene_99pct_brain1_inh = brain1_genes_df[inh_neuron_filter_brain1].quantile(pct)
gene_99pct_brain3_inh = brain3_genes_df[inh_neuron_filter_brain3].quantile(pct)

outlier_genes_99pct_inh = generate_mean_difference_plot(
                                    gene_99pct_brain1_inh,
                                    gene_99pct_brain3_inh, 
                                    '99th pct gene reads, brain1', 
                                    '99th pct gene reads, brain3',
                                    loa_std=2, data_transform='log', 
                                    title='inhibitory neurons')

### Flagged genes from log-transformed MD


In [118]:
print('Number of 99th percentile outlier genes, using log-transformed difference, in all cells:', 
      len(outlier_genes_99pct_log),'\n', sorted(outlier_genes_99pct_log))

print('\nNumber of 99th percentile outlier genes, using log-transformed difference, in neurons:', 
      len(outlier_genes_99pct_neurons),'\n', sorted(outlier_genes_99pct_neurons))

print('\nNumber of 99th percentile outlier genes, using log-transformed difference, in non-neuronal cells:', 
      len(outlier_genes_99pct_nn),'\n', sorted(outlier_genes_99pct_nn))

print('\nNumber of 99th percentile outlier genes, using log-transformed difference, in excitatory neurons:', 
      len(outlier_genes_99pct_exc),'\n', sorted(outlier_genes_99pct_exc))

print('\nNumber of 99th percentile outlier genes, using log-transformed difference, in inhibitory neurons:', 
      len(outlier_genes_99pct_inh),'\n', sorted(outlier_genes_99pct_inh))

In [134]:
outlier_genes_all = sorted(list(dict.fromkeys([*outlier_genes_99pct_log, 
                                               *outlier_genes_99pct_neurons, 
                                               *outlier_genes_99pct_nn, 
                                               *outlier_genes_99pct_exc, 
                                               *outlier_genes_99pct_inh])))
print(outlier_genes_all)

### TODO: Spatial expression plots of flagged genes

In [136]:
# Goal: Display spatial expression of outlier_genes_all from merfish data
brain1_adata = adata[adata.obs['codebook']==brain1_codebook]

example_sections_brain1 = ['1198980108', '1198980114']
example_sections_brain3 = ['1199651042', '1199651045']

def display_brain1_vs_brain3_expression(genes, **kwargs):
    """ Display the spatial expression of a requested list of genes for a 
    two example sections of brain1 and brain3 """
    

## Modified MD w/ difference as % of mean on y-axis

Another idea for how to handle the large range of values in gene reads is to divide the difference by the mean to plot the difference as a percentage of the mean, which is "useful when there is an increase in variability of the difference as teh magnitude of the measurement increases" (reference: Figure 7 in "Giavarina D. (2015). Biochemia Medica. 25(2): 141-51. doi:10.11613/BM.2015.015.")


In [142]:
# 95th percentile, all cells
pct = 0.95
gene_95pct_brain1 = brain1_genes_df.quantile(pct)
gene_95pct_brain3_neurons = brain3_genes_df.quantile(pct)

outlier_genes_95pct_diffAsPctOfMean = generate_mean_difference_plot(
                                        gene_95pct_brain1,
                                        gene_95pct_brain3, 
                                        '95th pct gene reads, brain1', 
                                        '95th pct gene reads, brain3',
                                        loa_std=2, 
                                        data_transform='diff_as_pct_of_mean')

# 99th percentile, all cells
pct = 0.99
gene_99pct_brain1 = brain1_genes_df.quantile(pct)
gene_99pct_brain3 = brain3_genes_df.quantile(pct)

outlier_genes_99pct_diffAsPctOfMean = generate_mean_difference_plot(
                                        gene_99pct_brain1,
                                        gene_99pct_brain3, 
                                        '99th pct gene reads, brain1', 
                                        '99th pct gene reads, brain3',
                                        loa_std=2, 
                                        data_transform='diff_as_pct_of_mean')

### Flagged genes from diff as % of mean

In [144]:
print('Number of 95th percentile outlier genes, using difference as a percent of the mean, in all cells:', 
      len(outlier_genes_95pct_diffAsPctOfMean),'\n', sorted(outlier_genes_95pct_diffAsPctOfMean))

print('\nNumber of 99th percentile outlier genes, using difference as a percent of the mean, in all cells:', 
      len(outlier_genes_99pct_diffAsPctOfMean),'\n', sorted(outlier_genes_99pct_diffAsPctOfMean))

# TODO: Extending MD plot analysis to >2 replicates

Eventually, we'll want to do this for more than just these two brains, Brain 1 & Brain 3. For example, the Imaging Dept. has a series of 12(?), sequential, ~10um sections of mouse brain that they've been using as "identical" replicates to test the consistency of the Xenium. It would be great to be able to do this type of MD comparison on all 12(?) sections to flag potential outliers! However, the combinatorics of individual, pairwise MD plots quickly blows up!!

Luckily, the limma R/Bioconducter software package extends the MD / Bland-Altman plot method to compare each replicate (aka sample/method/observers/reader) to the average of all other replicates by:

"A virtual array is constructed by averaging the log-expression value for all the samples other than the sample of interest, and then a mean-difference plot is made between the single array and the virtual array." (Ritchie, et al. (2015). Nucleic Acids Research. doi:10.1093/nar/gkv007)

This may not be the most efficient way to flag instances where entire sections failed, but it should suffice.
